### Thermodynamics
#### Unifac method - Activity coefficients

In [1]:
import numpy as np
import pandas as pd
import CoolProp.CoolProp as CP

Molecular group data, extracted from material.

In [118]:
groups = { # grupo: {secundário i:[molécula,R,Q],secundário i+1:[molécula,R,Q],etc}
    1: {1:['CH3',0.9011,0.848],2:['CH2',0.6744,0.540],3:['CH',0.4469,0.228],4:['C',0.2195,0]},
    2: {5:['CH2CH',1.3454,1.176],6:['CHCH',1.1167,0.867],7:['CH2C',1.1173,0.988],8:['CHC',0.8886,0.676],70:['CC',0.6605,0.485]},
    3: {9:['ACH',0.5313,0.400],10:['AC',0.3652,0.120]},
    4: {11:['ACCH3',1.2663,0.968],12:['ACCH2',1.0396,0.660],13:['ACCH',0.8121,0.348]},
    5: {14:['OH',1.000,1.200]},
    6: {15:['CH3OH',1.4311,1.432]},
    7: {16:['H2O',0.9200,1.400]},
    8: {17:['ACOH',0.8952,0.680]},
    9: {18:['CH3CO',1.6724,1.488],19:['CH2CO',1.4457,1.180]},
    10: {20:['CHO',0.9980,0.948]},
    11: {21:['CH3COO',1.9031,1.728],22:['CH2COO',1.6764,1.420]},
    12: {23:['HCOO',1.2420,1.188]}
}
g_groups = {}
for i in groups.values():
    g_groups.update(i)
    
a_mn = pd.read_csv('a-mn.csv', sep=';', index_col=0)
a_mn.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,0.00,8602.000,61.13,76.50,986.5,697.2,1318.0,1333.0,476.40,677.0,232.100,507.0,251.50,391.500,255.70,206.60,920.70,287.800
2,-35.36,0.000,38.81,74.15,524.1,787.6,270.6,526.1,182.60,448.8,37.850,333.5,214.50,240.900,163.90,61.11,749.30,2811.500
3,-11.12,3.446,0.00,167.00,636.1,637.4,903.8,1329.0,25.77,347.3,5.994,287.1,32.14,161.700,122.80,9049.00,648.20,-4.449
4,-69.70,-113.600,-146.80,0.00,803.2,603.3,5695.0,4.9,-52.10,586.8,5688.000,197.8,213.10,19.020,-49.29,23.50,663.20,52.800
5,156.40,457.000,89.60,25.82,0.0,-137.1,353.5,-259.7,84.00,-203.6,101.100,267.8,28.06,8.642,42.70,-3210.00,-52.39,170.000


**Main class and it's methods:**
* initializer
* r, q and l coefficients
* $\phi$ and $\theta$ factors
* $\gamma_c$ - combinatorial activity coefficient
* $\psi$ factor

In [363]:
class molecule():
    def __init__(self,molGroups):
        self.g = molGroups.keys()
        self.v = molGroups.values()
        self.groups = molGroups        
        
    def rql(self):
        z,r,q = 10,0,0        
        num = len(self.g)
        if num > 1:
            for i,k in zip(self.g,self.v):
                r += g_groups[i][1] * k
                q += g_groups[i][2] * k
        else:
            r = g_groups[tuple(self.g)[0]][1] * tuple(self.v)[0]
            q = g_groups[tuple(self.g)[0]][2] * tuple(self.v)[0]
        self.l = 0.5 * z * (r - q) - (r - 1)
        self.r,self.q = r,q
        return(self.r,self.q,self.l)
    
    def phi_theta(self, fluid):
        phi,theta = {},{}       
        phi[self] = self.r * x / (self.r * x + fluid.r * (1 - x))
        phi[fluid] = fluid.r * (1 - x) / (self.r * x + fluid.r * (1 - x))
        theta[self] = self.q * x / (self.q * x + fluid.q * (1 - x))
        theta[fluid] = fluid.q * (1 - x) / (self.q * x + fluid.q * (1 - x))
        self.phi, self.theta = phi[self], theta[self]
        fluid.phi, fluid.theta = phi[fluid], theta[fluid]
        return(self.phi, self.theta, fluid.phi, fluid.theta)
    
    def combinatorial(self, fluid):
        self.x, fluid.x = x, (1-x)
        self.gamma_c_ln = np.log(self.phi / self.x) + 5 * self.q * np.log(self.theta / self.phi) + \
                    self.l - (self.phi / self.x) * ( fluid.x * fluid.l + self.x * self.l )
        fluid.gamma_c_ln = np.log(fluid.phi / fluid.x) + 5 * fluid.q * np.log(fluid.theta / fluid.phi) + \
                    fluid.l - (fluid.phi / fluid.x) * ( self.x * self.l + fluid.x * fluid.l )
        self.gamma = np.exp(self.gamma_c_ln)
        fluid.gamma = np.exp(fluid.gamma_c_ln)
        return(self.gamma, self.gamma_c_ln, fluid.gamma, fluid.gamma_c_ln)    
        
    def psi_mod(self, fluid, pairs):
        self.p = tuple([(i,j) for i in pairs for j in pairs])
        print(self.p)
        a, psi = {}, {}
        for i in self.p:
            j = (i[0]-1,i[1]-1)
            a[i] = a_mn.iat[j]
            psi[i] = np.exp(-a_mn.iat[j] / T)
        self.a_k, self.a_v, self.a = a.keys(), a.values(), a
        self.psi_k, self.psi_v, self.psi = psi.keys(), psi.values(), psi
        return(self.a_k, self.a_v, self.a)
        return(self.psi_k, self.psi_v, self.psi)
    
    def X_mod(self, fluid):                
        den = sum([self.groups[k] for k in list(self.g)]) * self.x + \
              sum([fluid.groups[k] for k in list(fluid.g)]) * fluid.x
        X = []
        g_list = list(set([i for j in [list(self.g),list(fluid.g)] for i in j]))
        for k in g_list:
            nom = 0
            if k in list(self.g):
                nom += self.groups[k] * self.x
            if k in list(fluid.g):
                nom += fluid.groups[k] * fluid.x
            X.append(nom / den)      
        self.X = X
        return(self.X)
    
    def Theta_mod(self, fluid):
        g_list = list(set([i for j in [list(self.g),list(fluid.g)] for i in j]))
        nom = [np.array([g_groups[k][2] for k in g_list]) * x for x in np.transpose(self.X)]
        print(nom)
        den = sum(nom)
        self.Theta = nom / den
        return(self.Theta)
      
        
    
    def Gamma_k(self, fluid):
        g_list = list( set( list(self.g).append( list(fluid.g) ) ) )
        X, X2 = [], {}
        for i,j in enumerate(self.x):
            denom = sum(self.v) * self.x[i] + sum(fluid.v) * fluid.x[i]
            somador = 0
            denom_theta = 0
            for k in g_list:
                if k in list(self.g):
                    somador += self.groups[k] * self.x[i]
                if k in list(fluid.g):
                    somador += fluid.groups[k] * fluid.x[i]
                denom_theta += ( somador / denom ) * g_groups[k][2]
                X.append(somador / denom)
            #X2[self.x[i]] = X
            theta_2 = np.array(X) / denom_theta
            for n,k in enumerate(g_list):
                var = theta_2 * self.p[n*len(g_list):(n+1)*len(g_list)]
                var2.append(theta_2 * self.p[(n+1)*len(g_list):n*len(g_list)])
                g_groups[k][2] * ( 1 - np.log( sum(var) - sum(var2[i]/var2[n])  )   )

In [288]:
x = np.linspace(0.1,0.9,9) 
T = 323.15
methanol = molecule({15:1})
water = molecule({16:1})
print(methanol.rql(),water.rql())
methanol.phi_theta(water)
print(methanol.phi[1])
print(methanol.theta[1])
print(methanol.combinatorial(water)[1])
print(x[1])
print(water.phi[1])
print(water.theta[1])
print((1-x)[1])
methanol.combinatorial(water)
print(methanol.gamma[1])
print(water.gamma[1])
methanol.psi(water,[5,6])
print([(methanol.psi[i],i) for i in methanol.p])

(1.4311, 1.432, -0.43559999999999954) (0.92, 1.4, -2.3199999999999994)
0.279998434779
0.203640500569
[ 0.46249839  0.34127621  0.24475479  0.16890336  0.11045023  0.06671734
  0.03549577  0.01495076  0.00354866]
0.2
0.720001565221
0.796359499431
0.8
1.40674173734
1.0294603912


AttributeError: 'molecule' object has no attribute 'psi'

In [5]:
p = tuple([(i,j) for i in [5,6] for j in [5,6]])
p

((5, 5), (5, 6), (6, 5), (6, 6))

In [6]:
p = tuple([(i,j) for i in [5,6,7] for j in [5,6,7]])
print(p)

((5, 5), (5, 6), (5, 7), (6, 5), (6, 6), (6, 7), (7, 5), (7, 6), (7, 7))


In [202]:
a = list({1:2,3:4}.keys())
b = list({3:5,6:7}.keys())
a= [a,b]
print(a)
bb = {1:2,3:3,6:6}
a_list = list(set([i for j in a for i in j]))
print(a_list)
[[bb[k]*x,(1-bb[k])*(1-x)] for k in a_list]

[[1, 3], [3, 6]]
[1, 3, 6]


[[array([ 0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8]),
  array([-0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1])],
 [array([ 0.3,  0.6,  0.9,  1.2,  1.5,  1.8,  2.1,  2.4,  2.7]),
  array([-1.8, -1.6, -1.4, -1.2, -1. , -0.8, -0.6, -0.4, -0.2])],
 [array([ 0.6,  1.2,  1.8,  2.4,  3. ,  3.6,  4.2,  4.8,  5.4]),
  array([-4.5, -4. , -3.5, -3. , -2.5, -2. , -1.5, -1. , -0.5])]]

In [206]:
lista = [1,2]
lista = [lista, ([i for i in [1,2,3,4]])]
print(lista)
list(set([i for j in lista for i in j]))

[[1, 2], [1, 2, 3, 4]]


[1, 2, 3, 4]

In [364]:
T = 307
pentano = molecule({1:2,2:3})
acetona = molecule({1:1,18:1})
print(pentano.rql(),acetona.rql())

(3.8254, 3.316, -0.27839999999999865) (2.5735, 2.336, -0.3859999999999988)


#### pentane-ketone mixture - combinatorial
* r,q and l coefficients
* $\phi$ and $\theta$ factors
* $\gamma^c$ coefficient

In [365]:
acetona.rql()
acetona.phi_theta(pentano)
acetona.combinatorial(pentano)
print(acetona.phi[0],acetona.theta[0],pentano.phi[0],pentano.theta[0])
print(acetona.gamma[0],acetona.gamma_c_ln[0],pentano.gamma[0],pentano.gamma_c_ln[0])

0.381447799788 0.392377229416 0.618552200212 0.607622770584
0.98104397552 -0.0191379931836 0.987386125049 -0.0126941052623


#### residual
* psi

In [366]:
acetona.psi_mod(pentano,[1,1,9])
for i in acetona.psi_k:
    print(acetona.psi[i])

((1, 1), (1, 1), (1, 9), (1, 1), (1, 1), (1, 9), (9, 1), (9, 1), (9, 9))
0.916524826418
1.0
1.0
0.211868065424


In [367]:
acetona.X_mod(pentano)
#acetona.Theta_mod(pentano)
print(acetona.X[0][1])

0.420454545455


In [368]:
acetona.X

[array([ 0.42682927,  0.42045455,  0.40566038]),
 array([ 0.13414634,  0.10227273,  0.02830189]),
 array([ 0.43902439,  0.47727273,  0.56603774])]